# Vysegmentovanie TLE objektu z obrázka pomocou metódy prehovanie 

Inštalácia potrebných knižníc

In [ ]:
!pip install opencv-python

Import potrebných knižníc

In [ ]:
from glob import glob # return all file paths that match a specific pattern
import os             # provides functions for creating and removing a directory (folder)
import cv2            # image processing
import shutil         # save files locally

### Vystrihnutie objektu z obrázka na základe normovaných YOLO súradníc

Vlastná metóda na prepočítanie normovaných súradníc na pôvodný tvar

In [ ]:
def recalculate_normalized_coordinates(txt_in_dir):
    file = open(txt_in_dir, 'r')
    line = file.readline()
    file.close()
    _, x, y, width, height = line.split()
    #type(x)
    
    # the size of the original image
    bg_image = cv2.imread("/home/jovyan/data/lightning/LiviaMurankova/DATA/TP/images/TP_20140805_000233_ARBO.jpg")
    bg_h, bg_w, bg_channel = bg_image.shape
    #print(bg_h, bg_w, bg_channel)

    x=float(x)*bg_w
    y=float(y)*bg_h
    width=float(width)*bg_w
    height=float(height)*bg_h

    return [_, x, y, width, height]

Algoritmus na vystrihnutie objektu z obrázka

In [ ]:
imageName="TP_2014_Aug_04_225613_ARBO"

anot_out_dir="/home/jovyan/data/lightning/LiviaMurankova/DATA/new/New_TP/croped_events"
image = "/home/jovyan/data/lightning/LiviaMurankova/DATA/new/New_TP/images/" + imageName + ".jpg"
tpImage_txt_file = "/home/jovyan/data/lightning/LiviaMurankova/DATA/new/New_TP/txt/" + imageName + ".txt"

import cv2
from glob import glob
import random

#print(tpImage_txt_file)
coordinates = recalculate_normalized_coordinates(tpImage_txt_file)
x_center, y_center, box_width, box_height = int(coordinates[1]), int(coordinates[2]), int(coordinates[3]), int(coordinates[4])
x1 = int(x_center - box_width/2)
y1 = int(y_center - box_height/2)
x2 = int(x_center + box_width/2)
y2 = int(y_center + box_height/2)
#print(x1, y1, x2, y2)

img = cv2.imread(image)
crop_img = img[y1:y2, x1:x2]

annot_path_and_matching_img_name = anot_out_dir + '/' + imageName + '.jpg'
#print(annot_path_and_matching_img_name)
cv2.imwrite(annot_path_and_matching_img_name, crop_img)

### Vysegmentovanie TLE javu pomocou metódy prahovanie

In [ ]:
# defining the paths to the input images and the output images    
in_dir = '/home/jovyan/data/lightning/LiviaMurankova/DATA/new/New_TP/cropped_events'
out_dir = '/home/jovyan/data/lightning/LiviaMurankova/DATA/new/New_TP/cropped_events_without_background'

# read the input image file names with paths into a list
infiles = in_dir + '/*'
img_names = glob(infiles)

# loop over each input image in a for loop
for file_name in img_names:
    
    # read an image and convert to gray image
    im_gray = cv2.imread(file_name, 0)
    
    # image threshold with OTSU thresholding + getting the threshold value
    thresh, im_bw = cv2.threshold(im_gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    print("Thresh: ", thresh)
    
    if(thresh != 0):
    
        #print('processing: %s...' % file_name)

        # read an input image
        src = cv2.imread(file_name, 1)
        tmp = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)

        # image threshold according to the detected threshold value
        alpha = cv2.threshold(tmp, thresh, 255, cv2.THRESH_BINARY)[1]

        # transparent background
        b, g, r = cv2.split(src)
        rgba = [b, g, r, alpha]
        dst = cv2.merge(rgba, 4)

        # write the result to disk in the previously created output directory
        size = len(os.path.basename(file_name))
        name = os.path.basename(file_name)[:size - 3] + 'png'
        outfile = out_dir + '/' + name
        
        # save tresholded image
        cv2.imwrite(outfile, dst)